In [2]:
!pip install rasterio

!pip install cython
!pip install cartopy

  Using cached https://files.pythonhosted.org/packages/e5/92/fe8838fa8158931906dfc4f16c5c1436b3dd2daf83592645b179581403ad/Cartopy-0.17.0.tar.gz
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 /usr/local/lib/python3.6/dist-packages/pip/_vendor/pep517/_in_process.py get_requires_for_build_wheel /tmp/tmp2qxu4osc Check the logs for full command output.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import rasterio
import gdal
from scipy.io import loadmat
 
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import pandas as pd
import seaborn as sns
%matplotlib inline

In [6]:
dset_gtiff = "/content/drive/My Drive/Data/gpw_v4_population_count_rev11_2020_2pt5_min.tif"
gdal.AllRegister()
global ds
ds = gdal.Open(dset_gtiff)
band = ds.GetRasterBand(1)
arr = band.ReadAsArray()
ds_size = arr.shape
if band.GetMinimum() is None or band.GetMaximum() is None:
   band.ComputeStatistics(0)
      
print ("[ NO DATA VALUE ] = ", band.GetNoDataValue()) # none
print ("[ MIN ] = ", band.GetMinimum())
print ("[ MAX ] = ", band.GetMaximum())
print (ds_size)

ds.GetProjection()
dataGTIFF = ds.ReadAsArray()

[ NO DATA VALUE ] =  -3.4028230607370965e+38
[ MIN ] =  0.0
[ MAX ] =  1710353.125
(4320, 8640)


In [0]:
def gather_data():
  all_data = [loadmat('/content/drive/My Drive/Data/TrackData_RMM_phase' + str(i + 1) + '.mat') for i in range(8)]
  eight_phase = []
  for dset in all_data:
    dset_lat = [row for row in dset['latstore']]
    dset_long = [row for row in dset['longstore']]
    hur_path = [zip(dset_lat[a], dset_long[a]) for a in range(4000)]
    eight_phase.append(hur_path)
  return eight_phase
  
def hour_binning(hurr_phases, width = 1.25):
  all_data = []
  for phase in hurr_phases:
    hour_bins = np.zeros((int(90/width), int(280/width))) #setting up a 2d-array for binning.
    for hurricane in phase:
      hurricane = list(hurricane)
      hurricane = [coord for coord in hurricane if coord[0] != 0. and coord[1] != 0.]
      for i in range(len(hurricane[1:])):
        prev = hurricane[i-1]
        if check_same_bin(prev[0], prev[1], hurricane[i][0], hurricane[i][1], width):
          hour_bins[int(hurricane[i][0]/width)][int(hurricane[i][1] / width)] += 1
        else:
          split_hour(hour_bins, int(prev[0]/width), int(prev[1]/width), int(hurricane[i][0]/width), int(hurricane[i][1]/width))
    all_data.append(hour_bins)
  all_data = np.array([[[float('nan') if num < 1 else num for num in row] for row in data] for data in all_data])
  return all_data

def split_hour(hurr_phases, coord1x, coord1y, coord2x, coord2y):
  dx = coord2x - coord1x
  dy = coord2y - coord1y
  increment = 1 /(abs(dx) + abs(dy))
  incx, incy = 0,0
  hurr_phases[coord1x][coord1y] += increment
  while(dx != 0 or dy != 0):
      if(abs(dx) == abs(dy)):
        while(dx != 0 and dy !=0):
          incx += 1 if dx > 0 else -1
          incy += 1 if dy > 0 else -1
          hurr_phases[coord1x+incx][coord1y+incy] += increment
          dx -= 1 if dx > 0 else -1
          dy -= 1 if dy > 0 else -1
      else:
        if(abs(dx) > abs(dy)):
          incx += 1 if dx > 0 else -1
          dx -= 1 if dx > 0 else -1
          hurr_phases[coord1x+incx][coord1y+incy] += increment
        elif (abs(dy) > abs(dx)):
          incy += 1 if dy > 0 else -1
          dy -= 1 if dy > 0 else -1
          hurr_phases[coord1x+incx][coord1y+incy] += increment

def transform_coordinate(j, i, padfTransform):
  Yp = padfTransform[0] + i*padfTransform[1] + i*padfTransform[2];
  Xp = padfTransform[3] + j*padfTransform[4] + j*padfTransform[5];
  if Yp < 0:
    Yp += 360
  if Yp > 360:
    Yp -= 360
  return (Xp, Yp)

def population_binning(data, width = 1.25):
  pop_bins = np.zeros((int(90/width), int(280/width)))
  for i in range(len(data) // 2):
    for j in range(len(data[i])):
      coords = transform_coordinate(i, j, ds.GetGeoTransform())
      try:
        if (0 <= coords[0] <= 90) and (0 <= coords[1] <= 280) and data[i][j] >= 1:
          pop_bins[int(coords[0] / width)][int(coords[1] / width)] += round(data[i][j])
      except:
        print(coords)
  return pop_bins

def check_same_bin(x1, y1, x2, y2, width):
  return int(x1/width) == int(x2/width) and int(y1/width) == int(y2/width) 

def popxhours(population, hour_set):
  pop_hour_bins = [population*phase for phase in hour_set]
  pop_hour_bins = [[[float('nan') if np.isnan(num) or num <= 1 else num for num in row] for row in data] for data in pop_hour_bins]
  return np.array(pop_hour_bins)
  
def popxhours_anomaly(pop_hour_bins, percentage = False): 
  sum_pop_hours = np.zeros(pop_hour_bins.shape[1:])
  phases_participated = np.zeros(pop_hour_bins.shape[1:]) #Need to get the number of phases each bin has been in
  for i in range(sum_pop_hours.shape[0]):
    for j in range(sum_pop_hours.shape[1]):
      vals = [phase[i,j] for phase in pop_hour_bins if not np.isnan(phase[i,j])]
      sum_pop_hours[i,j] = np.nansum(vals)
      phases_participated[i,j] = len(vals)
  sum_pop_hours = np.array([[sum_pop_hours[i,j] / phases_participated[i,j] if phases_participated[i,j] > 0 else float('nan') for j in range(len(sum_pop_hours[i]))]for i in range(sum_pop_hours.shape[0])])
  all_data = []
  
  for phase in pop_hour_bins:
    phase_data = np.zeros(pop_hour_bins.shape[1:])
    for i in range(phase_data.shape[0]):
      for j in range(phase_data.shape[1]):
        if not percentage:
          phase_data[i,j] = float('nan') if np.isnan(sum_pop_hours[i,j]) else phase[i,j] - sum_pop_hours[i,j]
        elif percentage and phases_participated[i,j] == 8:
          phase_data[i,j] = float('nan') if np.isnan(sum_pop_hours[i,j]) else phase[i,j] - sum_pop_hours[i,j]
          phase_data[i,j] = phase_data[i,j] * 100 /sum_pop_hours[i,j]
        else:
          phase_data[i,j] = float('nan')
    all_data.append(phase_data)
  return all_data

def data_dict(data = None, label=None, xaxis=None, yaxis=None, cmapscheme=None, cmapmin=None, cmapmax=None, cmaplabel=None):
  ps = {}
  ps['data'] = data
  ps['label'] = label
  ps['xaxis'] = xaxis
  ps['yaxis'] = yaxis
  ps['cmapscheme'] = cmapscheme
  ps['cmapmin'] = cmapmin
  ps['cmapmax'] = cmapmax
  ps['cmaplabel'] = cmaplabel
  return ps

In [0]:
def plot_hours(hourset, mean = False):
  phase_plot = [1,2,3,4,5,6,7,8]
  hours = [np.nanmean(phase) if mean else np.nansum(phase) for phase in hourset]
  plt.plot(phase_plot, hours)
  plt.show()

import random
def bootstrap_data(TCphases, samp_size, num_samples, mean = False):
  phase_plot = [1,2,3,4,5,6,7,8]
  dist_data = [[] for i in range(8)]
  for sample in range(num_samples):
    small_phases = []
    for phase in TCphases:
      indicies = random.sample(range(0, len(phase)), samp_size)
      tempset = [phase[index] for index in indicies]
      small_phases.append(tempset)
    sample_bins = hour_binning(small_phases)
    for i in range(len(sample_bins)):
      dist_data[i].append(np.nansum(sample_bins[i]))#/samp_size))
    sum_phase = [np.nansum(phase) for phase in sample_bins]
    plt.plot(phase_plot, sum_phase, linewidth = 0.5)
  plt.show()
  sns.set(rc={'figure.figsize':(30,20)})
  for i in range(len(dist_data)):
    sns.distplot(np.array(dist_data[i]), hist=False, kde_kws={"label": "Phase " + str(i + 1)})
  return dist_data

def distribution_plot(TCphases):
  plt.figure(figsize=(30,20))
  positions = [i for i in range(1, 13)]
  labels = [10**i for i in positions]
  plt.xticks(positions, labels)
  for i in range(len(TCphases)):
    phase = TCphases[i].flatten()
    phase = phase[np.logical_not(np.isnan(phase))]
    sns.set()
    sns.distplot(phase,hist=False,kde=True, kde_kws={"label": "Phase " + str(i + 1)})

In [0]:
hour_set = hour_binning(gather_data())
hour_set_plots = data_dict(hour_set,'Hours Exposed', None, None, 'rainbow', np.nanmin(np.array(hour_set)), np.nanmax(np.array(hour_set)), 'Hours')

hour_anom = popxhours_anomaly(hour_set, True)
set_0 = abs(np.nanmax(np.array(hour_anom))) if abs(np.nanmax(np.array(hour_anom))) > abs(np.nanmin(np.array(hour_anom))) else abs(np.nanmin(np.array(hour_anom)))
hour_anomaly_plots = data_dict(hour_anom, 'Hours Exposed Anomaly', None, None, 'RdBu', -set_0, set_0, 'Percent Difference From Mean')

binned_pop = population_binning(dataGTIFF)
all_data = popxhours(binned_pop, hour_set)
all_data = np.log10(all_data)
pop_hours_data = data_dict(all_data,'Population-Hours', None, None, 'rainbow', np.nanmin(np.array(all_data)), np.nanmax(np.array(all_data)), 'Person-Hours')

anom = popxhours_anomaly(all_data, True)
set_0 = abs(np.nanmax(np.array(anom))) if abs(np.nanmax(np.array(anom))) > abs(np.nanmin(np.array(anom))) else abs(np.nanmin(np.array(anom)))
anomaly_dataset = data_dict(anom, 'Population Hour Anomaly', None, None, 'RdBu', -set_0, set_0, 'Percent Difference From Mean')

In [0]:
distribution_plot(binned_pop)